In [1]:
from pydantic import BaseModel, Field
from typing import List
import guidance
import os


In [2]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [3]:
base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-30B-A3B-Instruct-2507", base_url="http://127.0.0.1:30000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [4]:
# def run_gen_test(lm):
#     with guidance.user():
#         lm += "What is the capital of France? and its population?"
#         lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

#     with guidance.assistant():
#         lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
#         print(lm["answer"])

# run_gen_test(base_lm)

In [5]:
from pydantic import BaseModel, Field
from typing import List
import guidance
from guidance import models, system, user, assistant, json as gen_json, select
import json
from enum import Enum


class FirstPrinciples(BaseModel):
    absolute_truths: List[str] = Field(
        ...,
        description="Look at the problem statement and list the things that you know that are absolute truths."
    )


class GroundUpReasoning(BaseModel):
    """Model for the LLM-generated reasoning plan."""
    reason: str = Field(
        ...,
        description="Based on the known truths reason in a concise sharp manner from ground up to address the problem."
    )

class Reflection(BaseModel):
    """Model for an individual reasoning step execution."""
    evaluation: str = Field(
        ...,
        description="Reflect on your reasoning and identify gaps / uncertainties / mistakes."
    )

class ReasoningConclusion(BaseModel):
    """Model for the final reasoning conclusion."""
    conclusion: str = Field(
        ...,
        description="Address the identified gaps in the previous step if any and deduce the final conclusion."
    )

def dynamic_final_answer_schema(Options):
    class FinalAnswer(BaseModel):
        answer: Options = Field(..., description="Final answer.")

    return FinalAnswer


@guidance()
def _first_principles(llm):
    with assistant():
        llm += gen_json(
            name="first_principles",
            schema=FirstPrinciples,
            max_tokens=5000
        )
    return llm

@guidance()
def _generate_groundup_reasoning(llm):
    """
    Generate a custom reasoning plan tailored to the query.
    """
    with assistant():
        llm += gen_json(
            name="groundup_reasoning",
            schema=GroundUpReasoning,
            max_tokens=5000  # Reduced to encourage conciseness
        )
    return llm

@guidance()
def _reflection(llm):
    with assistant():
        llm += gen_json(
            name="reflection",
            schema=Reflection,
            max_tokens=5000
        )
    return llm
    

@guidance()
def _reasoning_conclusion(llm):
    with user():
        llm += """Based on the information, make logical deductions."""
    
    with assistant():
        llm += gen_json(
            name="reasoning_conclusion",
            schema=ReasoningConclusion,
            max_tokens=5000
        )

    return llm

@guidance
def dynamic_structured_reasoning(llm, query, output_choices):
    Options = Enum("Choices", {choice: choice for choice in output_choices})
    final_answer_schema = dynamic_final_answer_schema(Options)

    with system():
        llm += """You are an expert in problem solving. For any query, reason from first principles to solve the query. 
        Structure all outputs as JSON. Be very brief in your steps, tending towards concise but complete responses.
        """

    with user():
        llm += f"""Query: {query}
        Look at the problem statement and list the things that you know that are absolute truths -
        """

    llm += _first_principles()

    with user():
        llm += f"""
        Based on the known truths reason from ground up to address the problem, be extremely brief and to the point in your reasoning -
        """

    llm += _generate_groundup_reasoning()

    with user():
        llm += f"""
        Reflect to identify gaps / uncertainties / mistakes in reasoning process -
        """

    llm += _reflection()

    with user():
        llm += f"""
        Make logical deductions based on the information you have to arrive at a conclusion -
        """

    llm += _reasoning_conclusion()

    with user():
        llm += f"""
        Your final answer -
        """

    with assistant():
        llm += gen_json(
            name="answer",
            schema=final_answer_schema,
            max_tokens=1000)

    return llm

In [6]:
def generate(query, output_choices, lm = None):
    lm = lm or base_lm
    result_lm = lm + dynamic_structured_reasoning(
                    query=query,
                    output_choices = output_choices
                )

    answer = json.loads(result_lm.get("answer"))['answer']
    
    groundup_reasoning = json.loads(result_lm.get("groundup_reasoning"))['reason']
    first_principles = json.loads(result_lm.get("first_principles"))['absolute_truths']
    reflection = json.loads(result_lm.get("reflection"))['evaluation']   
    reasoning_conclusion = json.loads(result_lm.get("reasoning_conclusion"))['conclusion']
    return first_principles, groundup_reasoning, reflection, reasoning_conclusion, answer


In [7]:
from datasets import load_dataset

# Load the BBH dataset from Hugging Face
dataset = load_dataset("WildEval/ZebraLogic", 'mc_mode')



In [8]:
from json import JSONDecodeError


def evaluate_reasoning(row):
    """
    Placeholder function to evaluate your reasoning system on a single row.
    Replace with your dynamic_structured_reasoning function.
    """
    
    # from guidance.models.experimental import SglangModel
    # lm = SglangModel(
    #     model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    #     base_url="http://127.0.0.1:30000/v1",
    #     echo=True,
    #     api_key=os.environ.get("OPENAI_API_KEY", "NO_KEY"),
    # )
    puzzle = row['puzzle']
    question = row['question']
    query = puzzle + '\n\n' + question
    choices = row['choices']
    target = row['answer']
    print(row['id'])
    # Your reasoning system should return a predicted answer
    try:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = generate(query, choices)
    except JSONDecodeError as e:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = ["JSONDecodeError"]*5
    return {
        'correct': pred == target,
        'first_principles': first_principles,
        'groundup_reasoning': groundup_reasoning,
        'reflection': reflection,
        'reasoning_conclusion': reasoning_conclusion,
        'predicted_answer': pred, 
    }


In [9]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import pandas as pd

# # df is already df.iloc[600:650]  (keeps original index)
# def run_parallel_preserve_index(df, workers=16):
#     idx_to_row = df.to_dict(orient="index")  # {index: rowdict}
#     results = {}

#     with ThreadPoolExecutor(max_workers=workers) as ex:
#         futs = {ex.submit(evaluate_reasoning, row): idx
#                 for idx, row in idx_to_row.items()}
#         for fut in as_completed(futs):
#             idx = futs[fut]
#             results[idx] = fut.result()

#     res_df = pd.DataFrame.from_dict(results, orient="index")
#     # ensure order matches df (and keep original index)
#     res_df = res_df.reindex(df.index)
#     return df.join(res_df)

# df = pd.DataFrame(dataset['test'])
# # df.head()
# # CURRENTLY FINISHED - [:600]

# df = df.iloc[600:605]
# df = run_parallel_preserve_index(df, workers=16)


In [10]:
# import pandas as pd
# df = pd.DataFrame(dataset['test'])
# len(df)

In [12]:
import pandas as pd

df = pd.DataFrame(dataset['test'])
# df.head()
# CURRENTLY FINISHED - [:2300]

df = df.iloc[2300:2400]

res = df.apply(lambda r: pd.Series(evaluate_reasoning(r)), axis=1)
df = pd.concat([df, res], axis=1)

lgp-test-2x6-5#mc-8


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

lgp-test-4x6-7#mc-23
lgp-test-5x4-24#mc-4
lgp-test-5x5-7#mc-23
lgp-test-3x2-13#mc-2
lgp-test-3x5-3#mc-3
lgp-test-5x6-23#mc-10
lgp-test-2x3-5#mc-0
lgp-test-2x5-12#mc-7
lgp-test-2x2-14#mc-0
lgp-test-4x6-27#mc-12
lgp-test-5x5-36#mc-16
lgp-test-2x6-33#mc-4
lgp-test-5x2-18#mc-4
lgp-test-5x6-17#mc-6
lgp-test-2x4-1#mc-0
lgp-test-6x6-2#mc-35
lgp-test-3x2-4#mc-0
lgp-test-4x3-35#mc-2
lgp-test-6x4-20#mc-15
lgp-test-4x3-7#mc-9
lgp-test-2x3-23#mc-0
lgp-test-3x5-12#mc-12
lgp-test-6x5-25#mc-0
lgp-test-6x2-0#mc-11
lgp-test-6x6-0#mc-1
lgp-test-3x5-24#mc-12
lgp-test-4x5-2#mc-8
lgp-test-5x3-33#mc-1
lgp-test-4x5-23#mc-18
lgp-test-2x4-21#mc-1
lgp-test-6x4-21#mc-17
lgp-test-2x3-19#mc-4
lgp-test-5x6-24#mc-26
lgp-test-4x6-29#mc-7
lgp-test-3x5-10#mc-8
lgp-test-6x4-32#mc-7
lgp-test-4x6-19#mc-8
lgp-test-3x2-7#mc-5
lgp-test-6x3-34#mc-4
lgp-test-4x6-28#mc-1
lgp-test-3x6-19#mc-2
lgp-test-3x3-1#mc-2
lgp-test-2x5-18#mc-5
lgp-test-5x5-38#mc-0
lgp-test-2x6-19#mc-1
lgp-test-5x4-10#mc-2
lgp-test-2x5-37#mc-4
lgp-test-5x5-

In [13]:
df

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
2300,lgp-test-2x6-5#mc-8,"There are 2 houses, numbered 1 to 2 from left ...",What is Birthday of the person who lives in Ho...,"[april, sept]",sept,2024-07-03T21:21:31.304376,True,"[House 1 is occupied by the engineer., The eng...","Engineer is in House 1 (Clue 1), so Eric is in...",All steps follow logically from clues. No gaps...,The person in House 2 has a birthday in Septem...,sept
2301,lgp-test-4x6-7#mc-23,"There are 4 houses, numbered 1 to 4 from left ...",What is PhoneModel of the person who lives in ...,"[oneplus 9, iphone 13, samsung galaxy s21, goo...",iphone 13,2024-07-03T21:21:31.332278,True,[House 2 has the Dragonfruit smoothie (Clue 9)...,From Clue 8 and Clue 12: Black hair → February...,The reasoning is logically consistent and all ...,The person in House 4 uses an iPhone 13.,iphone 13
2302,lgp-test-5x4-24#mc-4,"There are 5 houses, numbered 1 to 5 from left ...",What is Name of the person who lives in House 2?,"[Eric, Arnold, Alice, Peter, Bob]",Bob,2024-07-03T21:21:31.346000,False,[The British person is in the third house (Clu...,"Alice is in house 3 (Clue 10, 2). British pers...",All steps are logically consistent. Key deduct...,The person in House 2 is Arnold.,Arnold
2303,lgp-test-5x5-7#mc-23,"There are 5 houses, numbered 1 to 5 from left ...",What is Food of the person who lives in House 5?,"[stir fry, pizza, grilled cheese, spaghetti, s...",stir fry,2024-07-03T21:21:31.349113,True,[House 4 has the person who loves stew (Clue 6...,"House 4: Peter, stew, modern. House 3: grilled...",Initial reasoning incorrectly assumed House 5 ...,"House 1: spaghetti, September, ? person, ? flo...",stir fry
2304,lgp-test-3x2-13#mc-2,"There are 3 houses, numbered 1 to 3 from left ...",What is Name of the person who lives in House 2?,"[Peter, Arnold, Eric]",Arnold,2024-07-03T21:21:31.306728,True,"[Peter is the person who enjoys gardening., Pe...",Peter is in house 3 and likes gardening. Eric ...,Reasoning is logically sound: Peter is in hous...,The person in House 2 is Arnold.,Arnold
...,...,...,...,...,...,...,...,...,...,...,...,...
2395,lgp-test-3x5-14#mc-12,"There are 3 houses, numbered 1 to 3 from left ...",What is Education of the person who lives in H...,"[high school, bachelor, associate]",high school,2024-07-03T21:21:31.312656,True,"[Arnold is not in House 3., Eric is not in Hou...","From clues: Eric ≠ House 3, Arnold ≠ House 3 ⇒...",Initial reasoning correctly deduces Peter is i...,The person in House 3 has a high school educat...,high school
2396,lgp-test-3x4-13#mc-1,"There are 3 houses, numbered 1 to 3 from left ...",What is Food of the person who lives in House 1?,"[pizza, grilled cheese, spaghetti]",grilled cheese,2024-07-03T21:21:31.309951,True,"[House 1 is on the far left, House 2 in the mi...",Eric = classical (Clue 3). Pizza lover is left...,Initial reasoning correctly deduced that class...,The food of the person in House 1 is grilled c...,grilled cheese
2397,lgp-test-5x3-7#mc-1,"There are 5 houses, numbered 1 to 5 from left ...",What is BookGenre of the person who lives in H...,"[fantasy, science fiction, biography, romance,...",romance,2024-07-03T21:21:31.340335,True,"[Eric loves biography books., The person who l...",House 1 has mountain vacation (clue 5). Arnold...,The reasoning is logically consistent and all ...,The person in House 1 has the book genre: roma...,romance
2398,lgp-test-4x4-4#mc-7,"There are 4 houses, numbered 1 to 4 from left ...",What is Hobby of the person who lives in House 2?,"[cooking, painting, photography, gardening]",painting,2024-07-03T21:21:31.322808,True,"[Peter's birthday is in February (Clue 1)., Pe...",House 2 has January birthday (Clue 4). Peter h...,The reasoning is logically consistent and full...,The hobby of the person in House 2 is painting.,painting


In [14]:
df['correct'].value_counts()

correct
True     79
False    21
Name: count, dtype: int64

In [15]:
path = './zebralogic_eval.csv'
df.to_csv(path, mode="a", index=False, header=not os.path.exists(path))

In [16]:
dub = pd.read_csv(path)


In [17]:
dub['correct'].value_counts()

correct
True     1743
False     657
Name: count, dtype: int64

In [ ]:
dub

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
0,lgp-test-6x4-37#mc-16,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 5?,"['Eric', 'Bob', 'Alice', 'Peter', 'Carol', 'Ar...",Bob,2024-07-03T21:21:31.375234,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1,lgp-test-3x6-15#mc-13,"There are 3 houses, numbered 1 to 3 from left ...",What is Education of the person who lives in H...,"['associate', 'high school', 'bachelor']",associate,2024-07-03T21:21:31.316147,True,"['Peter is in House 1.', 'Eric is in House 3.'...",Peter (House 1) has bachelor's degree → House ...,Initial reasoning correctly deduces: House 1: ...,House 3 has education: associate.,associate
2,lgp-test-4x6-24#mc-9,"There are 4 houses, numbered 1 to 4 from left ...",What is BookGenre of the person who lives in H...,"['science fiction', 'mystery', 'romance', 'fan...",mystery,2024-07-03T21:21:31.335059,True,"['House 1 has the bird keeper (Clue 2).', 'The...","House 1: red hair (Clue 6), bird (Clue 2). Hou...",Initial reasoning incorrectly assumed black ha...,The BookGenre of the person in House 2 is myst...,mystery
3,lgp-test-5x5-18#mc-0,"There are 5 houses, numbered 1 to 5 from left ...",What is Name of the person who lives in House 1?,"['Peter', 'Bob', 'Arnold', 'Eric', 'Alice']",Eric,2024-07-03T21:21:31.350746,True,['House 1 is occupied by the teacher (Clue 8)....,House 1 is the teacher (Clue 8). The only name...,"The reasoning correctly eliminates Bob, Alice,...",The person in House 1 is Eric.,Eric
4,lgp-test-4x2-35#mc-0,"There are 4 houses, numbered 1 to 4 from left ...",What is Name of the person who lives in House 1?,"['Eric', 'Arnold', 'Alice', 'Peter']",Peter,2024-07-03T21:21:31.319814,True,"['Eric is in the second house.', 'The Swedish ...","Alice is in house 3 (clue 5), so she is Britis...",The reasoning assumes Arnold is not in house 1...,The person in House 1 is Peter.,Peter
...,...,...,...,...,...,...,...,...,...,...,...,...
2295,lgp-test-2x4-13#mc-0,"There are 2 houses, numbered 1 to 2 from left ...",What is Name of the person who lives in House 1?,"['Eric', 'Arnold']",Eric,2024-07-03T21:21:31.300937,True,['There are 2 houses: House 1 (left) and House...,"From Clue 1: basketball → House 1, cat → House...",No gaps or mistakes. Reasoning correctly appli...,The person in House 1 is Eric.,Eric
2296,lgp-test-2x6-15#mc-8,"There are 2 houses, numbered 1 to 2 from left ...",What is Food of the person who lives in House 2?,"['pizza', 'grilled cheese']",pizza,2024-07-03T21:21:31.304967,True,"['There are 2 houses, numbered 1 (left) and 2 ...",From Clue 5: Engineer → Bella's parent. From C...,Step-by-step logic is sound: Clue 5 + Clue 1 l...,"House 1: Arnold (engineer, grilled cheese, pho...",pizza
2297,lgp-test-4x2-9#mc-7,"There are 4 houses, numbered 1 to 4 from left ...",What is Occupation of the person who lives in ...,"['engineer', 'doctor', 'artist', 'teacher']",artist,2024-07-03T21:21:31.318875,True,"['The person who is a teacher is in House 1.',...",House 1: Teacher (from clue 5). Engineer = Arn...,The reasoning is logically sound and consisten...,The occupation of the person in House 4 is art...,artist
2298,lgp-test-3x5-26#mc-14,"There are 3 houses, numbered 1 to 3 from left ...",What is Food of the person who lives in House 3?,"['spaghetti', 'pizza', 'grilled cheese']",grilled cheese,2024-07-03T21:21:31.313595,True,"['The Ford F-150 is not in House 1 (Clue 1).',...",Ford F-150 is in House 2 → Peter (Tesla) ≠ Hou...,Initial reasoning correctly deduces Ford F-150...,The food of the person in House 3 is grilled c...,grilled cheese
